In [ ]:
import torch
import pickle
import numpy as np
from tqdm import tqdm
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# !rm -rf Hierarchal-DreamRec

In [ ]:
import os
os.chdir('/content/gdrive/MyDrive')
# !git clone https://github.com/Arman-Salahshour/Hierarchal-DreamRec.git
os.chdir('Hierarchal-DreamRec/')

# Train the model with Hyper-parameter fine tuning.

In [ ]:
!python -u DreamRec_Movies.py --tune --data Arman --epoch 70 --timesteps 50 --lr 0.001 --beta_sche linear --w 2 --optimizer adam --diffuser_type mlp1 --random_seed 100

Running K-Fold tuning...
=== Tuning alpha ===
  -> Value: 0.1
    Fold 1/5
Epoch 000; Train loss: 7.4675; Time: 00: 00: 01
Epoch 001; Train loss: 6.7411; Time: 00: 00: 00
Epoch 002; Train loss: 6.4173; Time: 00: 00: 00
Epoch 003; Train loss: 6.2234; Time: 00: 00: 00
Epoch 004; Train loss: 5.9408; Time: 00: 00: 00
Epoch 005; Train loss: 5.5100; Time: 00: 00: 00
Epoch 006; Train loss: 4.9734; Time: 00: 00: 00
Epoch 007; Train loss: 4.4123; Time: 00: 00: 00
Epoch 008; Train loss: 3.9655; Time: 00: 00: 01
Epoch 009; Train loss: 3.6143; Time: 00: 00: 01
-------------------------- VAL PHASE --------------------------
Average Loss: 5.2747
HR@5: 0.4002, NDCG@5: 0.3297
HR@10: 0.4673, NDCG@10: 0.3519
HR@20: 0.5299, NDCG@20: 0.3680
Evaluation cost: 00: 00: 00
----------------------------------------------------------------
Epoch 010; Train loss: 3.3048; Time: 00: 00: 01
Epoch 011; Train loss: 3.0851; Time: 00: 00: 00
Epoch 012; Train loss: 2.8860; Time: 00: 00: 01
Epoch 013; Train loss: 2.6829; T

In [ ]:
class Metric:
    def __init__(self, name, values,):
        self.name = name
        self.values = values
        self.eval_dict = defaultdict(list)
        self.bestOne = None

    def find_max_one(self):
        best = -np.inf
        for key in self.eval_dict.keys():
            temp = max(self.eval_dict[key])
            if temp > best:
               self.bestOne = key
               best =  temp

    def find_min_one(self):
        best = np.inf
        for key in self.eval_dict.keys():
            temp = min(self.eval_dict[key])
            if temp < best:
               self.bestOne = key
               best =  temp

    def __str__(self):
        info = [
            f"Metric Name: {self.name}",
            f"Values: {self.values}",
            f"Evaluation Dictionary: {dict(self.eval_dict)}",
            f"Best One: {self.bestOne}",
        ]
        return "\n".join(info)

data = torch.load("/tune/metrics.dict")

for idx, metric in enumerate(data):
    print(f"Metric {idx + 1}:")
    print(f"  Name: {metric.name}")
    print(f"  Values: {metric.values}")
    print(f"  Evaluation Dictionary: {dict(metric.eval_dict)}")
    print(f"  Best One: {metric.bestOne}")
    print("-" * 40)
print(data[0].bestOne, data[1].bestOne, data[2].bestOne)

# Train the model with the best hyper-parameters

In [46]:
!python -u DreamRec_Movies.py --train --no-tune --data Arman --timesteps 50 --epoch 100 --lr 0.01 --beta_sche linear --w 2 --optimizer adamw --diffuser_type mlp1 --random_seed 100

Running single training...
Epoch 000; Train loss: 2.1415; Time: 00: 00: 01
Epoch 001; Train loss: 1.6162; Time: 00: 00: 01
Epoch 002; Train loss: 1.3337; Time: 00: 00: 01
Epoch 003; Train loss: 1.1448; Time: 00: 00: 01
Epoch 004; Train loss: 0.9766; Time: 00: 00: 01
Epoch 005; Train loss: 0.8869; Time: 00: 00: 01
Epoch 006; Train loss: 0.7897; Time: 00: 00: 01
Epoch 007; Train loss: 0.7348; Time: 00: 00: 01
Epoch 008; Train loss: 0.6713; Time: 00: 00: 01
Epoch 009; Train loss: 0.6273; Time: 00: 00: 01
-------------------------- VAL PHASE --------------------------
Average Loss: 1.0571
HR@5: 0.5911, NDCG@5: 0.5177
HR@10: 0.6474, NDCG@10: 0.5362
HR@20: 0.7003, NDCG@20: 0.5496
Evaluation cost: 00: 00: 00
----------------------------------------------------------------
Epoch 010; Train loss: 0.5973; Time: 00: 00: 01
Epoch 011; Train loss: 0.5741; Time: 00: 00: 01
Epoch 012; Train loss: 0.5357; Time: 00: 00: 01
Epoch 013; Train loss: 0.5098; Time: 00: 00: 01
Epoch 014; Train loss: 0.4906; T

In [ ]:
!python -u DreamRec_Gen.py  --no-tune --data Arman --timesteps 600 --lr 0.001 --beta_sche linear --w 2 --optimizer adam --diffuser_type mlp1 --random_seed 100

In [12]:
import pandas as pd
train_df = pd.read_pickle(os.path.join('./data/Arman', 'train_data.df'))


In [48]:
!python -u DreamRec_Movies.py --data Arman

([2743, 1719, 1303, 3499, 1480, 379, 1352, 3689, 3534, 1049, 1954, 1204, 21, 3035, 1949, 157, 3363, 2827, 1537, 2907], [0.9737650156021118, 0.013962713070213795, 0.011718894354999065, 0.0005509151378646493, 2.2897436338098487e-06, 7.790619349634653e-08, 3.986861329963176e-08, 2.8003707797097377e-08, 2.9394173761687625e-09, 2.8284765640762544e-09, 1.4211707366484916e-09, 3.422585548307211e-10, 2.6460356128410467e-10, 1.0724587884425318e-10, 2.9426003300692116e-11, 2.926994063789934e-11, 2.6344114389953432e-11, 2.4479565943757997e-11, 1.8650442301648695e-11, 7.32380388973608e-12])


In [45]:
row = 2
train_df.iloc[row].seq

[1201, 2028, 2947, 2366, 1240, 3418, 1214, 3702, 1036, 2951]

In [37]:
train_df.iloc[row].seq_genres

[8, 16, 8, 8, 9, 8, 11, 9, 8, 8]

In [38]:
train_df.iloc[row].next

np.int64(1954)